In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.8 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0605.csv		       Github
 a0608.csv		       heatmap529_UK.ipynb
 a0609.csv		       heatmap530.ipynb
 ccc_compiled.csv	       Heat_Map_Online_language.ipynb
 csv			       __MACOSX
 csv.zip		       mapping_UK
 dataframe_csv		       mapping_US
 dataframe_csv_EU	       non_violant1.csv
 dataframe_csv_UK	       non_violant2.csv
 dataframe_result_529_UK.csv   UK
 dataframe_result_530_UK.csv   violant.csv
 dataframe_result_531_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_csv


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct=pd.read_csv('a0529-a0611_UK/a0609.csv',encoding='ISO-8859-1')

In [8]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0605.csv		       Github
 a0608.csv		       heatmap529_UK.ipynb
 a0609.csv		       heatmap530.ipynb
 ccc_compiled.csv	       Heat_Map_Online_language.ipynb
 csv			       __MACOSX
 csv.zip		       mapping_UK
 dataframe_csv		       mapping_US
 dataframe_csv_EU	       non_violant1.csv
 dataframe_csv_UK	       non_violant2.csv
 dataframe_result_529_UK.csv   UK
 dataframe_result_530_UK.csv   violant.csv
 dataframe_result_531_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_csv


In [ ]:
###
#ct=pd.read_csv('a0601.csv')

In [9]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,18343191100_10158286958501101,Facebook,2020-06-09 23:59:54,2021-07-28 12:46:39,video,NaN,newsweek.com,In response to a tweet referring to racism as ...,In response to a tweet referring to racism as ...,1.0,https://www.newsweek.com/crossfit-ceo-steps-do...,https://www.facebook.com/18343191100/posts/101...,1404117,2.277108,1.0,222,61,19,11,28,21,2,14,0,1,46,20,40,6,9,14,9,22,0,2,40736,Newsweek,Newsweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The freedom to disagree.,True,2008-04-22 21:09:05,en,0,40736|10158286958501101,{},{},{}
1,1,111188856119651_700712523833945,Facebook,2020-06-09 23:59:52,2021-07-24 18:57:52,video,NaN,youtube.com,Former Vice President Job Biden left a video m...,From the joys of birth to the sadness of death...,2.0,https://youtu.be/XZhDcy-XvP0,https://www.facebook.com/groups/11118885611965...,4539,-1.720000,2.0,33,5,0,12,0,0,0,0,0,1,24,8,9,7,4,20,4,10,0,3,7934143,Democratic Voice for Biden/Harris 2024,Newsweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,The freedom to disagree.,False,2008-04-22 21:09:05,en,0,7934143|700712523833945,{},{},{}
2,2,2185993931653388_2550218238564287,Facebook,2020-06-09 23:59:51,2021-08-15 15:47:26,native_video,NaN,youtube.com,Former Vice President Job Biden left a video m...,"Today, George Floyd was laid to rest as Presid...",2.0,https://www.facebook.com/TheRecount/videos/309...,https://www.facebook.com/2185993931653388/post...,14598,11.017241,2.0,246,75,215,10,2,13,48,30,0,10,16,8,9,6,3,7,2,7,0,3,8557227,The Recount,TheRecount,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Daily political news that won't waste your tim...,True,2019-02-13 16:46:58,en,0,8557227|2550218238564287,{},{},{}
3,3,131459315949_10158124442440950,Facebook,2020-06-09 23:59:48,2021-08-31 22:09:19,video,NaN,cbsnews.com,Former Vice President Job Biden left a video m...,"Not only did they know each other, but a forme...",1.0,https://cbsn.ws/2YnrGTT,https://www.facebook.com/131459315949/posts/10...,6759370,22.805061,2.0,1811,17674,2074,11,1613,37,700,413,0,3,564,25,125,48,48,60,75,122,0,16,14655,CBS News,CBSNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Original reporting and trusted news with conte...,True,2009-09-11 05:17:44,en,0,14655|10158124442440950,{},{},{}
4,4,51642564435_10157738513099436,Facebook,2020-06-09 23:59:47,2021-08-31 10:39:50,link,Rev. Sharpton Calls Out NFL's Goodell During G...,sanfrancisco.cbslocal.com,During his fiery eulogy at Tuesdayâs funeral...,SHARPTON CALLS OUT NFL During his fiery eulogy...,1.0,https://cbsloc.al/2AiiRT2,https://www.facebook.com/51642564435/posts/101...,298580,4.241379,1.0,56,16,116,9,1,38,1,9,0,0,12,5,8,6,4,7,9,7,0,3,20335,KPIX CBS San Francisco Bay Area,CBSSanFrancisco,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Expect more from KPIX 5,True,2009-02-20 23:49:11,en,0,20335|10157738513099436,{'Houston': 1},{'Houston': 1},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25788,25788,116031465116295_3273138612738882,Facebook,2020-06-09 17:21:14,2021-09-03 16:1

In [10]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [ ]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [11]:
###
ct_ner = ct

In [12]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{},{}
1,{},{},{}
2,{},{},{}
3,{},{},{}
4,{'Houston': 1},{'Houston': 1},{}


In [13]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [ ]:
ct_ner['NER-msg']

0               []
1               []
2               []
3               []
4        [Houston]
           ...    
25788           []
25789           []
25790           []
25791           []
25792           []
Name: NER-msg, Length: 25793, dtype: object

In [ ]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [ ]:
ct_ner['NER']

0                                                       []
1                                                       []
2                                                       []
3                                                       []
4                                       [Houston, Houston]
                               ...                        
25788    [Goldsboro, USA, Los Angeles, North Carolina, ...
25789                                                   []
25790                           [Brooklyn, Manhattan, NYC]
25791                           [Brooklyn, Manhattan, NYC]
25792                               [Milwaukee, MILWAUKEE]
Name: NER, Length: 25793, dtype: object

In [ ]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [ ]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [ ]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,18343191100_10158286958501101,Facebook,2020-06-09 23:59:54,2021-07-28 12:46:39,video,NaN,newsweek.com,In response to a tweet referring to racism as ...,In response to a tweet referring to racism as ...,1.0,https://www.newsweek.com/crossfit-ceo-steps-do...,https://www.facebook.com/18343191100/posts/101...,1404117,2.277108,1.0,222,61,19,11,28,21,2,14,0,1,46,20,40,6,9,14,9,22,0,2,40736,Newsweek,Newsweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The freedom to disagree.,True,2008-04-22 21:09:05,en,0,40736|10158286958501101,[],[],[],[]
1,1,111188856119651_700712523833945,Facebook,2020-06-09 23:59:52,2021-07-24 18:57:52,video,NaN,youtube.com,Former Vice President Job Biden left a video m...,From the joys of birth to the sadness of death...,2.0,https://youtu.be/XZhDcy-XvP0,https://www.facebook.com/groups/11118885611965...,4539,-1.720000,2.0,33,5,0,12,0,0,0,0,0,1,24,8,9,7,4,20,4,10,0,3,7934143,Democratic Voice for Biden/Harris 2024,Newsweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,The freedom to disagree.,False,2008-04-22 21:09:05,en,0,7934143|700712523833945,[],[],[],[]
2,2,2185993931653388_2550218238564287,Facebook,2020-06-09 23:59:51,2021-08-15 15:47:26,native_video,NaN,youtube.com,Former Vice President Job Biden left a video m...,"Today, George Floyd was laid to rest as Presid...",2.0,https://www.facebook.com/TheRecount/videos/309...,https://www.facebook.com/2185993931653388/post...,14598,11.017241,2.0,246,75,215,10,2,13,48,30,0,10,16,8,9,6,3,7,2,7,0,3,8557227,The Recount,TheRecount,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Daily political news that won't waste your tim...,True,2019-02-13 16:46:58,en,0,8557227|2550218238564287,[],[],[],[]
3,3,131459315949_10158124442440950,Facebook,2020-06-09 23:59:48,2021-08-31 22:09:19,video,NaN,cbsnews.com,Former Vice President Job Biden left a video m...,"Not only did they know each other, but a forme...",1.0,https://cbsn.ws/2YnrGTT,https://www.facebook.com/131459315949/posts/10...,6759370,22.805061,2.0,1811,17674,2074,11,1613,37,700,413,0,3,564,25,125,48,48,60,75,122,0,16,14655,CBS News,CBSNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Original reporting and trusted news with conte...,True,2009-09-11 05:17:44,en,0,14655|10158124442440950,[],[],[],[]
4,4,51642564435_10157738513099436,Facebook,2020-06-09 23:59:47,2021-08-31 10:39:50,link,Rev. Sharpton Calls Out NFL's Goodell During G...,sanfrancisco.cbslocal.com,During his fiery eulogy at Tuesdayâs funeral...,SHARPTON CALLS OUT NFL During his fiery eulogy...,1.0,https://cbsloc.al/2AiiRT2,https://www.facebook.com/51642564435/posts/101...,298580,4.241379,1.0,56,16,116,9,1,38,1,9,0,0,12,5,8,6,4,7,9,7,0,3,20335,KPIX CBS San Francisco Bay Area,CBSSanFrancisco,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Expect more from KPIX 5,True,2009-02-20 23:49:11,en,0,20335|10157738513099436,[Houston],[Houston],[],[houston]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25788,25788,116031465116295_3273138612738882,Facebook,2020-06-09 17:2

In [ ]:
ct_ner['NER-1']=ct_ner['NER']

In [ ]:
ct_ner['NER-1'][1]

[]

In [ ]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 4.1 MB/s 


In [ ]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,18343191100_10158286958501101,Facebook,2020-06-09 23:59:54,2021-07-28 12:46:39,video,NaN,newsweek.com,In response to a tweet referring to racism as ...,In response to a tweet referring to racism as ...,1.0,https://www.newsweek.com/crossfit-ceo-steps-do...,https://www.facebook.com/18343191100/posts/101...,1404117,2.277108,1.0,222,61,19,11,28,21,2,14,0,1,46,20,40,6,9,14,9,22,0,2,40736,Newsweek,Newsweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The freedom to disagree.,True,2008-04-22 21:09:05,en,0,40736|10158286958501101,[],[],[],[],[]
1,1,111188856119651_700712523833945,Facebook,2020-06-09 23:59:52,2021-07-24 18:57:52,video,NaN,youtube.com,Former Vice President Job Biden left a video m...,From the joys of birth to the sadness of death...,2.0,https://youtu.be/XZhDcy-XvP0,https://www.facebook.com/groups/11118885611965...,4539,-1.720000,2.0,33,5,0,12,0,0,0,0,0,1,24,8,9,7,4,20,4,10,0,3,7934143,Democratic Voice for Biden/Harris 2024,Newsweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,The freedom to disagree.,False,2008-04-22 21:09:05,en,0,7934143|700712523833945,[],[],[],[],[]
2,2,2185993931653388_2550218238564287,Facebook,2020-06-09 23:59:51,2021-08-15 15:47:26,native_video,NaN,youtube.com,Former Vice President Job Biden left a video m...,"Today, George Floyd was laid to rest as Presid...",2.0,https://www.facebook.com/TheRecount/videos/309...,https://www.facebook.com/2185993931653388/post...,14598,11.017241,2.0,246,75,215,10,2,13,48,30,0,10,16,8,9,6,3,7,2,7,0,3,8557227,The Recount,TheRecount,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Daily political news that won't waste your tim...,True,2019-02-13 16:46:58,en,0,8557227|2550218238564287,[],[],[],[],[]
3,3,131459315949_10158124442440950,Facebook,2020-06-09 23:59:48,2021-08-31 22:09:19,video,NaN,cbsnews.com,Former Vice President Job Biden left a video m...,"Not only did they know each other, but a forme...",1.0,https://cbsn.ws/2YnrGTT,https://www.facebook.com/131459315949/posts/10...,6759370,22.805061,2.0,1811,17674,2074,11,1613,37,700,413,0,3,564,25,125,48,48,60,75,122,0,16,14655,CBS News,CBSNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Original reporting and trusted news with conte...,True,2009-09-11 05:17:44,en,0,14655|10158124442440950,[],[],[],[],[]
4,4,51642564435_10157738513099436,Facebook,2020-06-09 23:59:47,2021-08-31 10:39:50,link,Rev. Sharpton Calls Out NFL's Goodell During G...,sanfrancisco.cbslocal.com,During his fiery eulogy at Tuesdayâs funeral...,SHARPTON CALLS OUT NFL During his fiery eulogy...,1.0,https://cbsloc.al/2AiiRT2,https://www.facebook.com/51642564435/posts/101...,298580,4.241379,1.0,56,16,116,9,1,38,1,9,0,0,12,5,8,6,4,7,9,7,0,3,20335,KPIX CBS San Francisco Bay Area,CBSSanFrancisco,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Expect more from KPIX 5,True,2009-02-20 23:49:11,en,0,20335|10157738513099436,[Houston],[Houston],[],[houston],[houston]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25788,25788,116031465116295_327313861

In [ ]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [ ]:
!ls

dataframe0529_post.csv	dataframe0606_post.csv	    dataframe_mapping_0605.csv
dataframe0530_post.csv	dataframe_mapping_0529.csv  dataframe_mapping_0606.csv
dataframe0531_post.csv	dataframe_mapping_0530.csv  dataframe_mapping_0607.csv
dataframe0601_post.csv	dataframe_mapping_0531.csv  dataframe_mapping_0608.csv
dataframe0602_post.csv	dataframe_mapping_0601.csv  dataframe_mapping_0609.csv
dataframe0603_post.csv	dataframe_mapping_0602.csv  dataframe_mapping_0610.csv
dataframe0604_post.csv	dataframe_mapping_0603.csv  dataframe_mapping_0611.csv
dataframe0605_post.csv	dataframe_mapping_0604.csv


In [ ]:
ct_mapping=pd.read_csv('dataframe_mapping_0609.csv',encoding='ISO-8859-1')

In [ ]:
len(ct_mapping['City'])

3314

In [ ]:
len(ct_ner['NER-1'])

25793

In [ ]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,18343191100_10158286958501101,Facebook,2020-06-09 23:59:54,2021-07-28 12:46:39,video,NaN,newsweek.com,In response to a tweet referring to racism as ...,In response to a tweet referring to racism as ...,1.0,https://www.newsweek.com/crossfit-ceo-steps-do...,https://www.facebook.com/18343191100/posts/101...,1404117,2.277108,1.0,222,61,19,11,28,21,2,14,0,1,46,20,40,6,9,14,9,22,0,2,40736,Newsweek,Newsweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The freedom to disagree.,True,2008-04-22 21:09:05,en,0,40736|10158286958501101,[],[],[],[],[]
1,1,111188856119651_700712523833945,Facebook,2020-06-09 23:59:52,2021-07-24 18:57:52,video,NaN,youtube.com,Former Vice President Job Biden left a video m...,From the joys of birth to the sadness of death...,2.0,https://youtu.be/XZhDcy-XvP0,https://www.facebook.com/groups/11118885611965...,4539,-1.720000,2.0,33,5,0,12,0,0,0,0,0,1,24,8,9,7,4,20,4,10,0,3,7934143,Democratic Voice for Biden/Harris 2024,Newsweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,The freedom to disagree.,False,2008-04-22 21:09:05,en,0,7934143|700712523833945,[],[],[],[],[]
2,2,2185993931653388_2550218238564287,Facebook,2020-06-09 23:59:51,2021-08-15 15:47:26,native_video,NaN,youtube.com,Former Vice President Job Biden left a video m...,"Today, George Floyd was laid to rest as Presid...",2.0,https://www.facebook.com/TheRecount/videos/309...,https://www.facebook.com/2185993931653388/post...,14598,11.017241,2.0,246,75,215,10,2,13,48,30,0,10,16,8,9,6,3,7,2,7,0,3,8557227,The Recount,TheRecount,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Daily political news that won't waste your tim...,True,2019-02-13 16:46:58,en,0,8557227|2550218238564287,[],[],[],[],[]
3,3,131459315949_10158124442440950,Facebook,2020-06-09 23:59:48,2021-08-31 22:09:19,video,NaN,cbsnews.com,Former Vice President Job Biden left a video m...,"Not only did they know each other, but a forme...",1.0,https://cbsn.ws/2YnrGTT,https://www.facebook.com/131459315949/posts/10...,6759370,22.805061,2.0,1811,17674,2074,11,1613,37,700,413,0,3,564,25,125,48,48,60,75,122,0,16,14655,CBS News,CBSNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Original reporting and trusted news with conte...,True,2009-09-11 05:17:44,en,0,14655|10158124442440950,[],[],[],[],[]
4,4,51642564435_10157738513099436,Facebook,2020-06-09 23:59:47,2021-08-31 10:39:50,link,Rev. Sharpton Calls Out NFL's Goodell During G...,sanfrancisco.cbslocal.com,During his fiery eulogy at Tuesdayâs funeral...,SHARPTON CALLS OUT NFL During his fiery eulogy...,1.0,https://cbsloc.al/2AiiRT2,https://www.facebook.com/51642564435/posts/101...,298580,4.241379,1.0,56,16,116,9,1,38,1,9,0,0,12,5,8,6,4,7,9,7,0,3,20335,KPIX CBS San Francisco Bay Area,CBSSanFrancisco,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Expect more from KPIX 5,True,2009-02-20 23:49:11,en,0,20335|10157738513099436,[Houston],[Houston],[],[houston],"[Houston, Harris County, Texas, United States]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
ct_ner['NER-1'][12]

['Houston, Harris County, Texas, United States']

In [ ]:
ct_ner.to_csv('dataframe0609_post.csv')